In [ ]:
from pprint import pprint

In [ ]:
from pydantic import BaseModel

class Country(BaseModel):
    name: str
    answer: str


In [ ]:
f = open("./.api-key", "r")
API_KEY = f.read()
print(API_KEY)
f.close()

In [ ]:
from openai import OpenAI, pydantic_function_tool

client = OpenAI(api_key = "API_KEY")

In [ ]:
system_content = "あなたは優秀な歴史学者であり、地政学者です。\
    ユーザーの質問に対して、G20の各国・地域について、それぞれ簡潔に答えてください。\
    必ず「フランス、アメリカ、イギリス、ドイツ、日本、イタリア、カナダ、EU、アルゼンチン、オーストラリア、ブラジル、中国、インド、インドネシア、メキシコ、韓国、ロシア、サウジアラビア、南アフリカ、トルコ、AU」のそれぞれについて説明してください。\
    回答できない場合は「分からない」という説明にしてください。"
query = "主要産業"
completion = client.beta.chat.completions.parse(
    model="gpt-4o",
    messages=[
    {
        "role": "system",
        "content": system_content,
    },
    {
        "role": "user",
        "content": "{}について教えてください。".format(query),
    },
    ],
    tools=[pydantic_function_tool(Country)],
)


In [ ]:
pprint(completion.choices[0].message.tool_calls)

In [ ]:
print(len(completion.choices[0].message.tool_calls))
print(type(completion.choices[0].message))
print(type(completion.choices[0].message.tool_calls))


In [ ]:
texts = list(
  map(
    lambda x: x.function.parsed_arguments.answer,
    completion.choices[0].message.tool_calls,
  )
)

print(texts)

In [ ]:
embedding = client.embeddings.create(input=texts, model="text-embedding-3-small", dimensions=512)
vectors = list(map(lambda x: x.embedding, embedding.data))
pprint(vectors)

In [ ]:
import umap

reducer = umap.UMAP()
vec2 = reducer.fit_transform(vectors)
print(vec2)


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

result = cosine_similarity(vectors, vectors)
pprint(result.shape)
pprint(result)

In [ ]:
from sklearn.cluster import KMeans

cluster = KMeans(n_clusters=3, init="k-means++", random_state=0).fit(vectors)
pprint(cluster.labels_)

In [ ]:
import numpy as np
vec2_extend = np.hstack([vec2, cluster.labels_.reshape(vec2.shape[0], 1)])
print(vec2_extend.shape)
pprint(vec2_extend)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib_fontja

fig = plt.figure()
# ax = fig.add_subplot(projection='2d')
# ax.scatter(res[:, 0], res[:, 1], color='green')

# # 各点にラベルを表示
# countries = list(map(lambda x: x.function.parsed_arguments.name, completion.choices[0].message.tool_calls))
# for i, name in enumerate(countries):
#     ax.text(res[i, 0], res[i, 1], name, fontsize=8)

# plt.show()

# UMAPの結果を2次元プロット
color = ["red", "blue", "green"]
plt.scatter(vec2[:, 0], vec2[:, 1], s=5, c=vec2_extend[:, 2], cmap="Set1")

# 各点にラベルを表示
countries = list(map(lambda x: x.function.parsed_arguments.name, completion.choices[0].message.tool_calls))
line_x, line_y = np.array([]), np.array([])
for i, name in enumerate(countries):
    plt.annotate(name, (vec2[i, 0], vec2[i, 1]), fontsize=8)
    for j, _ in enumerate(countries):
        if i == j:
            continue
        if (result[i, j] < 0):
            continue
        line_x = np.append(line_x, vec2[i, 0])
        line_y = np.append(line_y, vec2[i, 1])
        line_x = np.append(line_x, vec2[j, 0])
        line_y = np.append(line_y, vec2[j, 1])
        if (result[i, j] < 0.6):
            continue
        if (result[i, j] < 0.7):
            plt.plot(line_x, line_y, ls='dotted', lw=0.3)
        else:
            plt.plot(line_x, line_y, ls='-', lw=1.1)
        line_x, line_y = np.array([]), np.array([])

plt.show()